<a href="https://colab.research.google.com/github/adwitaarora/ai-hw-lab/blob/main/AIHWPractical2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests

params = {
    'dataset': 'sem_eval_2018_task_1',
}

response = requests.get('https://datasets-server.huggingface.co/splits', params=params)

In [ ]:
pip install datasets

In [4]:
from datasets import list_datasets, load_dataset

In [ ]:
dataset = load_dataset('sem_eval_2018_task_1', 'subtask5.english')

In [6]:
dataset_train = dataset["train"]

In [ ]:
pip install pandas

In [8]:
import pandas as pd

In [9]:
df = dataset_train.data.to_pandas()

In [ ]:
pip install torch

In [11]:
pip install transformers

In [12]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random

In [13]:
df.columns

Index(['ID', 'Tweet', 'anger', 'anticipation', 'disgust', 'fear', 'joy',
       'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust'],
      dtype='object')

In [15]:
def change(x):
  if x == True:
    return 1
  else :
    return 0
emotions = ['anger','anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'sadness']
for em in emotions:
  df[em] = df[em].apply(lambda x : change(x))

In [16]:
df

,ID,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,sadness
0,2017-En-21441,“Worry is a down payment on a problem you may ...,0,1,0,0,0,0,1,0
1,2017-En-31535,Whatever you decide to do make sure it makes y...,0,0,0,0,1,1,1,0
2,2017-En-21068,@Max_Kellerman it also helps that the majorit...,1,0,1,0,1,0,1,0
3,2017-En-31436,Accept the challenges so that you can literall...,0,0,0,0,1,0,1,0
4,2017-En-22195,My roommate: it's okay that we can't spell bec...,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
6833,2017-En-21383,@nicky57672 Hi! We are working towards your hi...,0,0,0,0,0,0,0,0
6834,2017-En-41441,@andreamitchell said @berniesanders not only d...,0,1,0,0,0,0,0,0
6835,2017-En-10886,@isthataspider @dhodgs i will fight this guy! ...,1,0,1,0,0,0,0,0
6836,2017-En-40662,i wonder how a guy can broke his penis while h...,0,0,0,0,0,0,0,0


In [17]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case = True
    )

In [18]:
def b_tp(preds, labels):
  return sum([preds == labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_fp(preds, labels):
  return sum([preds != labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_tn(preds, labels):
  return sum([preds == labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_fn(preds, labels):
  return sum([preds != labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_metrics(preds, labels):
  preds = np.argmax(preds, axis = 1).flatten()
  labels = labels.flatten()
  tp = b_tp(preds, labels)
  tn = b_tn(preds, labels)
  fp = b_fp(preds, labels)
  fn = b_fn(preds, labels)
  b_accuracy = (tp + tn) / len(labels)
  b_precision = tp / (tp + fp) if (tp + fp) > 0 else 'nan'
  b_recall = tp / (tp + fn) if (tp + fn) > 0 else 'nan'
  b_specificity = tn / (tn + fp) if (tn + fp) > 0 else 'nan'
  return b_accuracy, b_precision, b_recall, b_specificity

In [73]:
def preprocessing(input_text, tokenizer):
    return tokenizer.encode_plus(
                          input_text,
                          add_special_tokens = True,
                          max_length = 32,
                          pad_to_max_length = True,
                          return_attention_mask = True,
                          return_tensors = 'pt'
                    )

def bert(text, labels):
  token_id = []
  attention_masks = []


  for sample in text:
    encoding_dict = preprocessing(sample, tokenizer)
    token_id.append(encoding_dict['input_ids']) 
    attention_masks.append(encoding_dict['attention_mask'])


  token_id = torch.cat(token_id, dim = 0)
  attention_masks = torch.cat(attention_masks, dim = 0)
  labels = torch.tensor(labels)
  val_ratio = 0.2
  batch_size = 16

  train_idx, val_idx = train_test_split(
      np.arange(len(labels)),
      test_size = val_ratio,
      shuffle = True,
      stratify = labels)

  train_set = TensorDataset(token_id[train_idx], 
                            attention_masks[train_idx], 
                            labels[train_idx])

  val_set = TensorDataset(token_id[val_idx], 
                          attention_masks[val_idx], 
                          labels[val_idx])

  # Prepare DataLoader
  train_dataloader = DataLoader(
              train_set,
              sampler = RandomSampler(train_set),
              batch_size = batch_size
          )

  validation_dataloader = DataLoader(
              val_set,
              sampler = SequentialSampler(val_set),
              batch_size = batch_size
          )
  model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
  )

  optimizer = torch.optim.AdamW(model.parameters(), 
                                lr = 5e-5,
                                eps = 1e-08
                                )

  # Run on GPU
  model.cuda()
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  epochs = 10

  for _ in trange(epochs, desc = 'Epoch'):
      
      
      model.train()
      
      tr_loss = 0
      nb_tr_examples, nb_tr_steps = 0, 0

      for step, batch in enumerate(train_dataloader):
          batch = tuple(t.to(device) for t in batch)
          b_input_ids, b_input_mask, b_labels = batch
          optimizer.zero_grad()
        
          train_output = model(b_input_ids, 
                              token_type_ids = None, 
                              attention_mask = b_input_mask, 
                              labels = b_labels)
        
          train_output.loss.backward()
          optimizer.step()
        
          tr_loss += train_output.loss.item()
          nb_tr_examples += b_input_ids.size(0)
          nb_tr_steps += 1

    
      model.eval()

    
      val_accuracy = []
      val_precision = []
      val_recall = []
      val_specificity = []

      for batch in validation_dataloader:
          batch = tuple(t.to(device) for t in batch)
          b_input_ids, b_input_mask, b_labels = batch
          with torch.no_grad():
          
            eval_output = model(b_input_ids, 
                                token_type_ids = None, 
                                attention_mask = b_input_mask)
          logits = eval_output.logits.detach().cpu().numpy()
          label_ids = b_labels.to('cpu').numpy()

          b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits, label_ids)
          val_accuracy.append(b_accuracy)
          if b_precision != 'nan': val_precision.append(b_precision)
          if b_recall != 'nan': val_recall.append(b_recall)
          if b_specificity != 'nan': val_specificity.append(b_specificity)

      print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
      print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
      print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
      print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
      print('\t - Validation Specificity: {:.4f}\n'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '\t - Validation Specificity: NaN')
  return model



In [20]:
def prediction(model, new_sentence, c_class):
    test_ids = []
    test_attention_mask = []

    encoding = preprocessing(new_sentence, tokenizer)

    test_ids.append(encoding['input_ids'])
    test_attention_mask.append(encoding['attention_mask'])
    test_ids = torch.cat(test_ids, dim = 0)
    test_attention_mask = torch.cat(test_attention_mask, dim = 0)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    with torch.no_grad():
      output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

    prediction = f'{c_class}' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'Not Detected'

    # print('Input Sentence: ', new_sentence)
    # print('Predicted Class: ', prediction)
    return prediction

In [74]:
text = df.Tweet.values
labels = df.anger.values
model_anger = bert(text, labels)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceCla


	 - Train loss: 0.4525
	 - Validation Accuracy: 0.8140
	 - Validation Precision: 0.8042
	 - Validation Recall: 0.6884
	 - Validation Specificity: 0.8936



Epoch:  20%|██        | 2/10 [01:31<06:05, 45.63s/it]


	 - Train loss: 0.2729
	 - Validation Accuracy: 0.8001
	 - Validation Precision: 0.8276
	 - Validation Recall: 0.6026
	 - Validation Specificity: 0.9256



Epoch:  30%|███       | 3/10 [02:16<05:18, 45.51s/it]

...


	 - Train loss: 0.0536
	 - Validation Accuracy: 0.8154
	 - Validation Precision: 0.7699
	 - Validation Recall: 0.7466
	 - Validation Specificity: 0.8671



Epoch:  90%|█████████ | 9/10 [06:48<00:45, 45.30s/it]


	 - Train loss: 0.0463
	 - Validation Accuracy: 0.8132
	 - Validation Precision: 0.7861
	 - Validation Recall: 0.6989
	 - Validation Specificity: 0.8841



Epoch: 100%|██████████| 10/10 [07:33<00:00, 45.40s/it]


	 - Train loss: 0.0453
	 - Validation Accuracy: 0.8169
	 - Validation Precision: 0.7697
	 - Validation Recall: 0.7403
	 - Validation Specificity: 0.8660



In [23]:
text = df.Tweet.values
labels = df.anticipation.values
model_anticipation = bert(text, labels)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at


	 - Train loss: 0.4031
	 - Validation Accuracy: 0.8532
	 - Validation Precision: 0.2000
	 - Validation Recall: 0.0032
	 - Validation Specificity: 0.9967



Epoch: 100%|██████████| 2/2 [01:30<00:00, 45.19s/it]


	 - Train loss: 0.3303
	 - Validation Accuracy: 0.8547
	 - Validation Precision: 0.4000
	 - Validation Recall: 0.0108
	 - Validation Specificity: 0.9974



In [24]:
text = df.Tweet.values
labels = df.disgust.values
model_disgust = bert(text, labels)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceCla


	 - Train loss: 0.5253
	 - Validation Accuracy: 0.7914
	 - Validation Precision: 0.7480
	 - Validation Recall: 0.7033
	 - Validation Specificity: 0.8527



Epoch: 100%|██████████| 2/2 [01:36<00:00, 48.49s/it]


	 - Train loss: 0.3392
	 - Validation Accuracy: 0.7151
	 - Validation Precision: 0.5798
	 - Validation Recall: 0.9224
	 - Validation Specificity: 0.5948



In [25]:
text = df.Tweet.values
labels = df.fear.values
model_fear = bert(text, labels)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceCla


	 - Train loss: 0.3543
	 - Validation Accuracy: 0.8961
	 - Validation Precision: 0.7074
	 - Validation Recall: 0.6407
	 - Validation Specificity: 0.9494



Epoch: 100%|██████████| 2/2 [01:36<00:00, 48.50s/it]


	 - Train loss: 0.2400
	 - Validation Accuracy: 0.8292
	 - Validation Precision: 0.5195
	 - Validation Recall: 0.7888
	 - Validation Specificity: 0.8364



In [26]:
text = df.Tweet.values
labels = df.joy.values
model_joy = bert(text, labels)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceCla


	 - Train loss: 0.4455
	 - Validation Accuracy: 0.8365
	 - Validation Precision: 0.7728
	 - Validation Recall: 0.7871
	 - Validation Specificity: 0.8646



Epoch: 100%|██████████| 2/2 [01:31<00:00, 45.77s/it]


	 - Train loss: 0.2519
	 - Validation Accuracy: 0.8328
	 - Validation Precision: 0.7703
	 - Validation Recall: 0.7806
	 - Validation Specificity: 0.8642



In [27]:
text = df.Tweet.values
labels = df.love.values
model_love = bert(text, labels)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceCla


	 - Train loss: 0.2900
	 - Validation Accuracy: 0.9033
	 - Validation Precision: 0.7333
	 - Validation Recall: 0.0897
	 - Validation Specificity: 0.9968



Epoch: 100%|██████████| 2/2 [01:30<00:00, 45.33s/it]


	 - Train loss: 0.2082
	 - Validation Accuracy: 0.9128
	 - Validation Precision: 0.6258
	 - Validation Recall: 0.3782
	 - Validation Specificity: 0.9716



In [28]:
text = df.Tweet.values
labels = df.optimism.values
model_optimism = bert(text, labels)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceCla


	 - Train loss: 0.4877
	 - Validation Accuracy: 0.8038
	 - Validation Precision: 0.7867
	 - Validation Recall: 0.4321
	 - Validation Specificity: 0.9509



Epoch: 100%|██████████| 2/2 [01:30<00:00, 45.34s/it]


	 - Train loss: 0.3224
	 - Validation Accuracy: 0.7936
	 - Validation Precision: 0.7941
	 - Validation Recall: 0.3877
	 - Validation Specificity: 0.9616



In [29]:
text = df.Tweet.values
labels = df.sadness.values
model_sadness = bert(text, labels)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceCla


	 - Train loss: 0.5145
	 - Validation Accuracy: 0.7783
	 - Validation Precision: 0.6915
	 - Validation Recall: 0.4427
	 - Validation Specificity: 0.9146



Epoch: 100%|██████████| 2/2 [01:30<00:00, 45.27s/it]


	 - Train loss: 0.3830
	 - Validation Accuracy: 0.7842
	 - Validation Precision: 0.7056
	 - Validation Recall: 0.4542
	 - Validation Specificity: 0.9216



In [47]:
model_to_emotion = {model_anger : 'anger', model_anticipation : 'anticipation', model_disgust : 'disgust', model_fear : 'fear', model_joy : 'joy', model_love : 'love', model_optimism : 'optimism', model_sadness : 'sadness'}

In [48]:
def emo(sentence):
  emotions = []
  for model in model_to_emotion.keys():
    pred = prediction(model, sentence, model_to_emotion[model])
    if pred != 'Not Detected':
      emotions.append(pred)
  for emo in emotions:
    print(emo)

In [49]:
sentence = "this is making me scared and anxious"
emo(sentence)

fear


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [50]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
import torch
models = [model_anger, model_anticipation, model_disgust, model_fear, model_joy, model_love, model_optimism, model_sadness]
for model in model_to_emotion.keys():
  torch.save(model, f'/content/drive/MyDrive/models/{model_to_emotion[model]}.pt')

In [52]:
import os
directory = '/content/drive/MyDrive/models/'
i = 0
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        models[i] = torch.load(f)
        i+=1

In [56]:
sentence = "i love this sauce"
emo(sentence)

'love'

In [57]:
dataset_test = dataset["test"]

In [58]:
df_test = dataset_test.data.to_pandas()

In [76]:
text = df_test.Tweet.values
labels = df_test.anger.values
token_id = []
attention_masks = []


for sample in text:
  encoding_dict = preprocessing(sample, tokenizer)
  token_id.append(encoding_dict['input_ids']) 
  attention_masks.append(encoding_dict['attention_mask'])

token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)

val_ratio = 0.98
batch_size = 16

train_idx, val_idx = train_test_split(
    np.arange(len(labels)),
    test_size = val_ratio,
    shuffle = True,
    stratify = labels)

train_set = TensorDataset(token_id[train_idx], 
                          attention_masks[train_idx], 
                          labels[train_idx])

val_set = TensorDataset(token_id[val_idx], 
                        attention_masks[val_idx], 
                        labels[val_idx])

# Prepare DataLoader
train_dataloader = DataLoader(
            train_set,
            sampler = RandomSampler(train_set),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_set,
            sampler = SequentialSampler(val_set),
            batch_size = batch_size
        )

model = model_anger

val_accuracy = []
val_precision = []
val_recall = []
val_specificity = []
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for batch in validation_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    with torch.no_grad():
    
      eval_output = model(b_input_ids, 
                          token_type_ids = None, 
                          attention_mask = b_input_mask)
    logits = eval_output.logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits, label_ids)
    val_accuracy.append(b_accuracy)
    if b_precision != 'nan': val_precision.append(b_precision)
    if b_recall != 'nan': val_recall.append(b_recall)
    if b_specificity != 'nan': val_specificity.append(b_specificity)
# print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
print('\t - Validation Specificity: {:.4f}\n'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '\t - Validation Specificity: NaN')
# return model



/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


	 - Validation Accuracy: 0.8221
	 - Validation Precision: 0.7408
	 - Validation Recall: 0.7168
	 - Validation Specificity: 0.8777

